# Projeto 1 - Ciência dos Dados

Nome: Bernardo Hersbach Nagem Sarcinelli

Nome: Daniel Martins de Freitas 

Nome: Maria Eduarda Alves Rocha

Atenção: Serão permitidos grupos de até 4 pessoas, mas com uma rubrica mais exigente. 

___
Carregando algumas bibliotecas:

In [405]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
from IPython.display import display
import re

In [406]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
c:\Users\berna\C dados projeto1\cdados


Carregando a base de dados com as mensagens dos seus arquivos:

In [407]:
train = pd.read_csv('dados_treino_TRIO_mariaear.csv')
train.head(5)

,Comment,Topic
0,"Hey, have you heard about floating point error...",Physics
1,Are you numb?,Physics
2,thank you!!,Chemistry
3,"just what I was hoping for, thank you!",Chemistry
4,I thought it said dck and immediately thought ...,Chemistry


In [408]:
test = pd.read_csv('dados_teste_TRIO_mariaear.csv')
test.head(5)

,Comment,Topic
0,Well we learned IUPAC it was just never tested...,Chemistry
1,ok what happen when i lick licky or when crazy...,Chemistry
2,"Yeah, but 300 trillion is the total global car...",Physics
3,If only the particles outside the event horizo...,Physics
4,Here's a fun video describing more properties ...,Physics


___
## Classificador automático


Faça aqui uma descrição do seu assunto e o contexto referente aos rótulos cujas mensagens (ou reviews) deverão ser classificadas.

CAsos de seleção quimica e fisica

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [409]:
'''Conjunto de stopwords a serem removidas visando melhorar as estimativas'''
stopwords1=["i", "me", "my", "myself", "we", "our", "ours", "ourselves", "you", "your", "yours", "yourself", "yourselves", "he", "him", "his", "himself", "she", "her", "hers", "herself", "it", "its", "itself", "they", "them", "their", "theirs", "themselves", "what", "which", "who", "whom", "this", "that", "these", "those", "am", "is", "are", "was", "were", "be", "been", "being", "have", "has", "had", "having", "do", "does", "did", "doing", "a", "an", "the", "and", "but", "if", "or", "because", "as", "until", "while", "of", "at", "by", "for", "with", "about", "against", "between", "into", "through", "during", "before", "after", "above", "below", "to", "from", "up", "down", "in", "out", "on", "off", "over", "under", "again", "further", "then", "once", "here", "there", "when", "where", "why", "how", "all", "any", "both", "each", "few", "more", "most", "other", "some", "such", "no", "nor", "not", "only", "own", "same", "so", "than", "too", "very", "s", "t", "can", "will", "just", "don", "should", "now"]
def cleanup(text):
    """
        Função de limpeza muito simples que troca alguns sinais básicos por espaços
    """
    
    punctuation = '[´"!-.:?;$'']' # Note que os sinais [] são delimitadores de um conjunto.
    pattern = re.compile(punctuation)
    pattern2 = re.compile("\n")
    text_subbed = re.sub(pattern, '', text)
    text_subbed = re.sub(pattern2, ' ', text_subbed)
    return text_subbed

def cleaning(text):
    '''Outra função de de limpesa que separa mais alguns sibolos'''

    pontuacao='`^´~!#$:;.,[]{}’“”<>()/?'
    for i in pontuacao:
        if i in text:
            text=text.replace(i," ")

    text=text.lower()
    return text

def linha(text):
    '''Função que remove as stopwords do texto'''
    text=text.split()
    for k in range(len(text)):
        for stop in stopwords1:
            if text[k]==stop:
                text[k]=""
        for j in range(len(text[k])-1):
            text2=text[k]
            if text2[j]+text2[j+1]=="\n":
                text2=text2.replace(k[j],"$")
    text=",".join(text)
    text=text.replace(","," ")
    return text


In [410]:
test["comment_clean"]=test.Comment.apply(cleaning)
test["comment_clean"]=test.comment_clean.apply(linha)

train["comment_clean"]=train.Comment.apply(cleaning)
train["comment_clean"]=train.comment_clean.apply(linha)

quimica=train.loc[train.Topic=="Chemistry",["Topic","comment_clean"]]
fisica=train.loc[train.Topic=="Physics",["Topic","comment_clean"]]
quimica
fisica


,Topic,comment_clean
0,Physics,hey heard floating point errors seems like ...
1,Physics,numb
6,Physics,removed
8,Physics,force holding cars friction object travel...
10,Physics,i'm curious units mev u know electron-volt...
...,...,...
3489,Physics,viruses well crowds
3491,Physics,consider vertical cryogenic liquid storage ta...
3496,Physics,thanks mate really great
3497,Physics,teacher said object reach 0º kelvin ma...


In [411]:
# preparando texto para contagem
# quimica
quimica_string=""
for comentario in quimica.comment_clean:
    quimica_string+=" "+comentario
quimica_lista=quimica_string.split()
# fisica
fisica_string=""
for comentario1 in fisica.comment_clean:
    fisica_string+=" "+comentario1
fisica_lista=fisica_string.split()
serie_quimica=pd.Series(quimica_lista)
serie_fisica=pd.Series(fisica_lista)
# contagem
tabela_quimica=serie_quimica.value_counts()
quimica_total=tabela_quimica.sum()
tabela_fisica=serie_fisica.value_counts()
fisica_total=tabela_fisica.sum()
fisica_total
tabela_fisica

would             317
energy            244
like              215
time              209
physics           194
                 ... 
counterbalance      1
ladder              1
"paradox"           1
weighted            1
thickness           1
Name: count, Length: 8172, dtype: int64

In [412]:
todas_palavra=quimica_string+fisica_string
todas_palavra=todas_palavra.split()
serie_total=pd.Series(todas_palavra)
tabela_total=serie_total.value_counts()
tabela_total_relativa=serie_total.value_counts(True)

In [413]:
def classifica_frase(string):
    '''Função que calcula a probabilidade da frase ser de química ou fisica utilizando Naïve Bayes e suavisação de Laplace'''
    prob_Frase_dadoQ=1
    prob_Frase_dadoF=1
    string=string.lower()
    string=cleaning(string)
    frase=string.split()
    probQ=tabela_quimica.sum()/tabela_total.sum()
    probF=tabela_fisica.sum()/tabela_total.sum()

    #Calcula probabilidades Navie Bayes com suavisação de Laplace

    for i in frase:
        if i in tabela_quimica:
            prob_Frase_dadoQ=prob_Frase_dadoQ*((tabela_quimica[i]+1)/(tabela_quimica.sum()+len(tabela_total)))
        else:
            prob_Frase_dadoQ=prob_Frase_dadoQ*((0+1)/(tabela_quimica.sum()+len(tabela_total)))

        if i in tabela_fisica:
            prob_Frase_dadoF=prob_Frase_dadoF*((tabela_fisica[i]+1)/(tabela_fisica.sum()+len(tabela_total)))
        else:
            prob_Frase_dadoF=prob_Frase_dadoF*((0+1)/(tabela_fisica.sum()+len(tabela_total)))

    #Calcula a probabilidade da frase pertencer a quimíca ou fisíca
    prob_Q_dado_frase=prob_Frase_dadoQ*probQ
    prob_F_dado_frase=prob_Frase_dadoF*probF
    lista=[prob_Q_dado_frase, prob_F_dado_frase]
    return lista


In [414]:
def texto(string):
    '''Função que retorna a diciplina em que é mais provaval que a frase pertença'''
    lista1=classifica_frase(string)
    if lista1[0]>lista1[1]:
        return "Chemistry"
    elif lista1[1]>lista1[0]:
        return "Physics"

In [415]:
def teste(li,tabela):
    '''Função que retorna taxa de acerto em um determinado teste'''
    resultado=[]
    true=0
    false=0
    for item in li:
        testado=texto(item)
        resultado.append(testado)
        for comment in tabela.comment_clean:
            
            if comment==item:
                verdadeiro=(tabela[tabela["comment_clean"]==comment]["Topic"].values[0])
        
        if testado==verdadeiro:
            true+=1
        elif testado!=verdadeiro:
            false+=1
    total=len(resultado)
    porcentagem=total/100
    pacerto=true/porcentagem
    tabela["resultado"]=resultado
    return pacerto

In [416]:
test.dtypes

Comment          object
Topic            object
comment_clean    object
dtype: object

___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

In [417]:
lista_teste=[]
for i in test.comment_clean:
    lista_teste.append(i)

In [418]:
#Mostra taxa de acerto e tabela de Rópicos X Resultado no teste: Frases de treinamento
print(teste(lista_teste,test),'%')
pd.crosstab(test.resultado,test.Topic)

78.46666666666667 %


Topic,Chemistry,Physics
resultado,,
Chemistry,711,159
Physics,107,472


In [419]:
lista_teste1=[]
for i in train.comment_clean:
    lista_teste1.append(i)
    

In [420]:
#Mostra taxa de acerto e tabela de Rópicos X Resultado no teste: Frases de teste
print(teste(lista_teste1,train),'%')
pd.crosstab(train.resultado,train.Topic)

88.51428571428572 %


Topic,Chemistry,Physics
resultado,,
Chemistry,1780,158
Physics,104,1322


___
### Concluindo

Base de raciocínio: Tabela de exemplo em https://www.ibm.com/br-pt/topics/naive-bayes (seção Avaliando seu classificador Naïve Bayes)

Adotando Química como referencial de 'Positivo':
- Verdadeiros Positivos: Eram química e o classificador indicou ser química
       Coluna: Química    Linha: Química   Valor: 703
       Percentual: 711/Total x 100 = 711/1500 x 100 = 47.4%

- Falsos Positivos: Eram física mas o classificador indicou ser química
       Coluna: Física  Linha: Química  Valor: 160
       Percentual: 159/Total x 100 = 159/1500 x 100 = 10.6%

- Falsos Negativos: Eram química mas o classificador indicou ser física
       Coluna: Química  Linha: Física  Valor: 32
       Percentual: 107/Total x 100 = 107/1500 x 100 = 7.133%

- Verdadeiros Negativos: Eram física e o classificador indicou ser física
       Coluna: Física  Linha: Física  Valor: 467
       Percentual: 472/Total x 100 = 467/1500 x 100 = 31.466%

- Acurácia: A acurácia é a proporção de previsões corretas em relação ao total de previsões feitas (Fonte: https://www.cursospm3.com.br/blog/naive-bayes/) == Soma dos percentuais Verdadeiros [VP% (Verdadeiramente Química) + VN% (Verdadeiramente Física)]
       Acurácia = 47.4% + 31.46% = 78.86%

### Porque não se-pode utilizar as mensagens classificadas pelo classificador como amostra de treinamento 

O uso das frases classificadas pelo próprio classificador se mostra ruim como treinamento visto que ao ser uma frase de tópico supostamente desconhecido cria-se um risco da propagação de erros. Por exemplo se uma frase classificada de forma incorreta como física, porém faz parte do grupo de química, palavras pertencentes ao vocabulário de química podem ser adicionadas ao vocabulário de física ou termos pouco comuns podem se tornar mais comuns no vocabulário de física. No caso de múltiplas indicações incorretas isso pode reduzir a acurácia.   

### Naïve Bayes: Outras aplicações 

Como visto no projeto Classificadores Naïve Bayes conseguem identificar com boa confiabilidade assuntos relacionados a textos. Dessa forma é possível identificar diversas utilidades para essa ferramenta como:
1.	Um sistema de recomendação de posts, baseado nos gostos do usuário 
2.	Um sistema para avaliar comentários de um produto como positivo ou negativo

Em ambos os casos o algoritmo deve identificar o conteúdo do texto, porém a aplicação a partir da classificação se desenvolve de forma diferente, enquanto o primeiro exemplo poderia classificar os posts em assuntos e comparar novos posts a posts com os quais o usuário já interagiu para determinar sua relevância, o segundo pode ser utilizado para filtrar separar os comentários entre categorias e verificar se a avaliação de um produto é positiva, neutra ou negativa, por exemplo:

•	Se o classificador identificar 70% positivas e 30% negativas, pode-se assumir que os clientes avaliam como um bom produto;

•	Mas caso essas proporções sejam muito próximas no caso 52% e 48%, pode se interpretar como avalições neutras.
Em ambos os  casos a base de dados deve ser estudada de forma atenciosa pra evitar falsos positivos além de vocabulários característicos as classes utilizadas vizando melhrar a classificação.


### Limitações

Embora os resultados sejam positivos com 78% de acurácia no treinamento o modelo apresenta limitações visto que das 1500 frases presentes no teste o algoritmo fez 266 indicações incorretas e não foi capaz de determinar a qual tópico uma frase pertencia em 51 casos totalizando 317 falhas.

Pontos de melhoria filtrar mais palavras de pouca ou nenhuma relevância além das stopwords, valorizar mais palavras características de cada vocabulário no caso química ou física outra solução é o relaxamento da suposição de independência. “(...)pesquisas recentes tentam melhorar sua performance pelo relaxamento da suposição de independência. Friedman e Goldzmidt (1996) propuseram o Augmented Naive Bayes (Naive Bayes aumentado), que pode expressar relações parciais de dependência entre atributos.’’(Aloísio Carlos de Pina, Gerson Zaverucha, Melhorando a Performance do Algoritmo Naive Bayes para Regressão Através da Combinação de Atributos, p1)

Referencia: https://www.cos.ufrj.br/~ines/enia07_html/pdf/28095.pdf


___
### Qualidade do Classificador a partir de novas separações das mensagens entre Treinamento e Teste

Caso for fazer esse item do Projeto

___
## Aperfeiçoamento:

Leia atentamente a rubrica colocada no enunciado do Projeto 1 (última página). <br>
Trabalhos que conseguirem pelo menos conceito B vão evoluir em conceito dependendo da quantidade de itens avançados:

* IMPLEMENTOU outras limpezas e transformações que não afetem a qualidade da informação contida nas mensagens, mas tendem a melhorar na classificação das mensagens. Ex: stemming, lemmatization, stopwords.
* CONSIDEROU arquivo com três categorias na classificação das variáveis (OBRIGATÓRIO PARA QUARTETOS, sem contar como item avançado)
* CONSTRUIU o cálculo das probabilidades corretamente utilizando bigramas E apresentou referência sobre o método utilizado.
* EXPLICOU porquê não pode usar novas mensagens classificadas pelo próprio classificador como amostra de treinamento
* PROPÔS diferentes cenários para Naïve Bayes fora do contexto do projeto (pelo menos dois cenários diferentes, exceto aqueles já apresentados em sala pelos professores: por exemplo, filtro de spam)
* REFLETE criticamente sobre os resultados obtidos, identificando limitações do modelo e sugerindo possíveis melhorias ou diferentes abordagens com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa).
* DOCUMENTOU bem o código, com explicações claras para cada etapa do processo, incluindo o raciocínio por trás das decisões de modelagem e das transformações de dados.


___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[Natural Language Processing (Part 17)-Laplacian Smoothing](https://medium.com/@Coursesteach/natural-language-processing-part-17-laplacian-smoothing-7d4be71d0ded) **Mais simples**